In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
SQLALCHEMY_DATABASE_URL = "postgresql://robot-startml-ro:pheiph0hahj1Vaif@" \
                          "postgres.lab.karpov.courses:6432/startml"
engine = create_engine(SQLALCHEMY_DATABASE_URL)

In [3]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 500
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn,
                                       chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
        print(chunk_dataframe)
    conn.close()
    data = pd.concat(chunks, ignore_index=True)

    return data

In [4]:
def load_features() -> pd.DataFrame:
    query = "v_patrakeev_all_posts"
    query2 = "v_patrakeev_all_users"
    return batch_load_sql(query), batch_load_sql(query2)

In [5]:
#posts, users = load_features()
posts, users = pd.read_csv("post_feature_to_SQL_new", sep=";"),pd.read_csv("user_feature_to_SQL_new", sep=";")

In [6]:
posts.head()

,post_id,Topic,text_length,likes,posts_views,likes_share
0,7319,10.733667,790,720.0,6585.0,10.0
1,7318,10.733667,728,680.0,6785.0,10.0
2,7317,10.733667,636,731.0,6803.0,10.0
3,7316,10.733667,800,677.0,6572.0,10.0
4,7315,10.733667,803,2619.0,16940.0,15.0


In [7]:
users.head()

,user_id,age,City,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,168552,16,0.101590,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
1,168551,38,0.159781,0.138777,0.250000,0.400000,0.125000,0.000000,0.000000,0.000000,30.0,0.166667
2,168550,41,0.107315,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
3,168549,18,0.109604,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,34.0,0.029412
4,168548,36,0.099558,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435


Service operation scheme

In [20]:
#1 input id, time, limit, spisok_posts
from datetime import datetime
id = 10001
spisok_posts = list(posts.post_id) #count all posts
limit = 10
time = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #time recome

In [23]:
import hashlib
def get_exp_group(user_id: int) -> str:
    part = int(hashlib.md5((str(user_id) + 'salt').encode()).hexdigest(),16)% 100
    if part >= 50:
        exp_group = 'test'
    else: exp_group = 'control'
    return exp_group

In [11]:
#3 the list of posts user has seen
user_action_view = list(pd.read_sql(
     f"""
     select *
     from public.feed_data
     where user_id = {id} AND action = 'view'
         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )['post_id'])

In [12]:
#4 the list of all posts
all_posts = list(pd.read_sql(
     f"""
     select id
     from public.posts
         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )['id'])

In [13]:
#5 the list of posts user hasn't seen
spisok_posts = list(set(all_posts) - set(user_action_view))

In [14]:
#6 Create draft dataframe for preds
df = pd.DataFrame({'user_id': id,
                   'post_id': spisok_posts,
                   'time': time})

In [15]:
#7 add our features from download early
df = df.merge(posts,how='left',left_on='post_id', right_on='post_id')
df = df.merge(users,how='left',left_on='user_id', right_on='user_id')
df.head()

,user_id,post_id,time,Topic,text_length,likes,posts_views,likes_share,age,City,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,10001,1,2023-10-11 12:25:06,10.513725,1967,1067.0,8508.0,12.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
1,10001,2,2023-10-11 12:25:06,10.513725,2701,637.0,7495.0,8.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
2,10001,3,2023-10-11 12:25:06,10.513725,3408,1122.0,8412.0,13.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
3,10001,4,2023-10-11 12:25:06,10.513725,1026,1171.0,8158.0,14.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
4,10001,5,2023-10-11 12:25:06,10.513725,889,1153.0,8583.0,13.0,26,0.141995,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435


In [34]:
#8 download our save model
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
import joblib, pickle

model_LogReg = joblib.load('model_first_LogReg.pkl')
model_catboost = CatBoostClassifier()
model_catboost = model_catboost.load_model('model_second_catboost', format="cbm")

In [25]:
exp_group = get_exp_group(user_id)
if exp_group == 'control':
  recommendations = model_control.apply(data)
elif exp_group == 'test':
  recommendations = model_test.apply(data)
else:
  raise ValueError('unknown group')

In [15]:
from sklearn.linear_model import LogisticRegression
import joblib, pickle
model = joblib.load('model_first_LogReg.pkl')

In [26]:
model.predict_proba(df.drop(columns=['user_id','post_id', 'time']))

array([[9.99932473e-01, 6.75270730e-05],
       [9.99949896e-01, 5.01038979e-05],
       [9.99932053e-01, 6.79474518e-05],
       ...,
       [5.93305924e-01, 4.06694076e-01],
       [6.04457736e-01, 3.95542264e-01],
       [6.02774217e-01, 3.97225783e-01]])

In [28]:
df.drop(columns=['user_id','post_id', 'time'])

,Topic,text_length,likes,posts_views,likes_share,age,City,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,10.513725,1967,1067.0,8508.0,12.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
1,10.513725,2701,637.0,7495.0,8.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
2,10.513725,3408,1122.0,8412.0,13.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
3,10.513725,1026,1171.0,8158.0,14.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
4,10.513725,889,1153.0,8583.0,13.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6391,10.733667,803,2619.0,16940.0,15.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
6392,10.733667,800,677.0,6572.0,10.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
6393,10.733667,636,731.0,6803.0,10.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
6394,10.733667,728,680.0,6785.0,10.0,46,0.213787,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324


In [29]:
#9 pred
pred = pd.DataFrame(model.predict_proba(df.drop(columns=['user_id','post_id', 'time']))[:, 1], columns=["predict"])

In [30]:
#add our
df = pd.concat([df, pred], axis=1)

In [31]:
df = df.sort_values(by=["predict"], ascending=False)

In [32]:
spisok = tuple(df["post_id"].head(10).tolist())
spisok

(5406, 1359, 1301, 1160, 1085, 1025, 1018, 1235, 956, 7260)

In [33]:
post_top5_spisok = pd.read_sql(
    f"""
     select *
     from public.post
     where id IN {spisok}

         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )


In [34]:
post_top5_spisok

,id,text,topic
0,956,Clarke to press on with ID cards\n\nNew Home S...,politics
1,1018,Muslim police stops more likely\n\nUK Muslims ...,politics
2,1025,EU fraud clampdown urged\n\nEU member states a...,politics
3,1085,UK set to cut back on embassies\n\nNine overse...,politics
4,1160,BAA support ahead of court battle\n\nUK airpor...,politics
5,1235,Clarke faces ID cards rebellion\n\nCharles Cla...,politics
6,1301,Tory backing for ID cards\n\nThe Tories are to...,politics
7,1359,Asylum children to face returns\n\nThe UK gove...,politics
8,5406,Tobe Hooper (fresh off mainstream success with...,movie
9,7260,There are many different versions of this one ...,movie
